In [ ]:
#conda install -c conda-forge xyz2mol

In [ ]:
# Common imports
import numpy as np
from numpy import sqrt 
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pylab
import plotly.express as ex
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from sklearn.impute import SimpleImputer
from statsmodels.multivariate.pca import PCA

In [ ]:
import numpy as np 
import pandas as pd
df = pd.read_csv('/Users/abhisaurav/Desktop/comm_monophosphine.csv')

In [ ]:
df.drop(df.iloc[:, 0:5], inplace=True, axis=1)
df_X = df

In [ ]:
def plot_missing_values(df):
    """ For each column with missing values plot proportion that is missing."""
    data = [(col, df[col].isnull().sum() / len(df)) 
            for col in df.columns if df[col].isnull().sum() > 0]
    col_names = ['column', 'percent_missing']
    missing_df = pd.DataFrame(data, columns=col_names).sort_values('percent_missing')
    pylab.rcParams['figure.figsize'] = (15, 8)
    missing_df.plot(kind='barh', x='column', y='percent_missing'); 
    plt.title('Percent of missing values in colummns');

In [ ]:
count_missing = df_X.isnull().sum()
percent_missing = df_X.isnull().sum()/ len(df_X)
missing_value_df = pd.DataFrame({'column_name': df_X.columns,
                                 'count_missing': count_missing,
                                 'percent_missing': percent_missing}

missing_value_df = missing_value_df[missing_value_df['percent_missing']>0]
missing_value_df

In [ ]:
# Libraries

from sklearn.preprocessing import StandardScaler
from statsmodels.multivariate.pca import PCA 
import seaborn as sns
import seaborn as sns
import matplotlib.pyplot as plt
import pylab
import plotly.express as ex
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import xlsxwriter
%matplotlib inline

In [ ]:
# PCA with correlation matrix of data - Nipals

pc1 = PCA(df_X,
         ncomp=4,
         standardize=True,
         demean=True,
         normalize=False,
         gls=False,
         weights=None,
         method='nipals', 
         missing='fill-em')

In [ ]:
df_comp = pc1.loadings.T
df_comp

In [ ]:
X1_factors = pc1.factors
X1_factors

In [ ]:
correlation = X1_factors.corr()
correlation

In [ ]:
sns.heatmap(X1_factors.corr())
plt.show()

In [ ]:
# PCA with covariance matrix of data - Nipals

pc2 = PCA(df_X,
         ncomp=4,
         standardize=False,
         demean=True,
         normalize=False,
         gls=False,
         weights=None,
         method='nipals',
         missing='fill-em')

In [ ]:
df_comp = pc2.loadings.T
df_comp

In [ ]:
X2_factors = pc2.factors
X2_factors

In [ ]:
correlation = X2_factors.corr()
correlation

In [ ]:
sns.heatmap(X2_factors.corr())
plt.show()

In [ ]:
from statsmodels.multivariate.factor import Factor
model = Factor(X2_factors).fit()
model.plot_scree()
plt.show()

In [ ]:
explained_variance=pc1.eigenvals
explained_variance

In [ ]:
explained_variance_ratio = explained_variance/np.sum(explained_variance)
explained_variance_ratio

In [ ]:
(sns
 .lineplot(x=np.arange(1, 5),
           y=explained_variance_ratio,
           marker="o")
 .set(title="PVE vs. Principal Component",
      xlabel="Principal Component",
      ylabel="Proportion of Variance Explained",))

In [ ]:
# Cumulative PVE vs. Principal Component plot
(sns
 .lineplot(x=np.arange(1, 5),
           y=np.cumsum(explained_variance_ratio),
           marker="o")
 .set(title="Cumulative PVE vs. Principal Component",
      xlabel="Principal Component",
      ylabel="Cumulative Proportion of Variance Explained"))

In [ ]:
#with plt.style.context('dark_background'):
plt.figure(figsize=(6, 4))

plt.bar(range(4), explained_variance_ratio, alpha=0.5, align='center',label='individual explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal components')
plt.legend(loc='best')
plt.tight_layout()

In [ ]:
mono = pd.read_csv('/Users/abhisaurav/Desktop/comm_monophosphine.csv')

In [ ]:
mono_pca=mono.join(X2_factors)
mono_comm = mono_pca[mono_pca['com']==1]
#mono_comm=mono_comm.assign(size=lambda x: abs(x.comp_0))
mono_comm

In [ ]:
import numpy as np 
import pandas as pd
import plotly.express as px
# For plotting with matplotlib
import matplotlib.pyplot as plt
# For plotting with seaborn
import seaborn as sns  

In [ ]:
color_discrete = {"N": 'green', "Y": 'red'}

fig = px.scatter_3d(mono_comm, x='comp_0', y='comp_1', z='comp_2',
                    color = 'symbol',
                    color_discrete_map=color_discrete,
                    text = 'check',
                    hover_name = 'Kraken ID',
                    size = 'vbur_vbur_min',
                    size_max=30)
#fig.show()
fig.update_layout(margin=dict(l=10, r=10, b=10, t=10))

Cluster Analysis

In [ ]:
df_X

In [ ]:
mono_cluster=df_X.join(X2_factors)
mono_cluster

In [ ]:
# This normalization will be performed just for PC1 and PC2, but can be done for all the components.
scale1 = 1.0/(max(mono_cluster['comp_0']) - min(mono_cluster['comp_0']))
scale2 = 1.0/(max(mono_cluster['comp_1']) - min(mono_cluster['comp_1']))
scale3 = 1.0/(max(mono_cluster['comp_2']) - min(mono_cluster['comp_2']))
scale4 = 1.0/(max(mono_cluster['comp_3']) - min(mono_cluster['comp_3']))


In [ ]:
# And we add the new values to our PCA table
mono_cluster['PC0_normalized']=[i*scale1 for i in mono_cluster['comp_0']]
mono_cluster['PC1_normalized']=[i*scale2 for i in mono_cluster['comp_1']]
mono_cluster['PC2_normalized']=[i*scale3 for i in mono_cluster['comp_2']]
mono_cluster['PC3_normalized']=[i*scale4 for i in mono_cluster['comp_3']]
mono_cluster.head(10)

In [ ]:
from sklearn.cluster import KMeans
wcss = []
for i in range(1,11):
   model = KMeans(n_clusters = i, init = "k-means++")
   model.fit(mono_cluster)
   wcss.append(model.inertia_)
plt.plot(range(1,11), wcss)
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
#Reference : https://chem-workflows.com/articles/2019/07/02/exploring-the-chemical-space-by-pca/

from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm


range_n_clusters = [2, 3, 4, 5, 6]
for n_clusters in range_n_clusters:
    fig, (ax1,ax2)= plt.subplots(1, 2)
    fig.set_size_inches(8, 4)
    
    kmeans = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = kmeans.fit_predict(mono_cluster[['PC0_normalized','PC1_normalized','PC2_normalized','PC3_normalized']])
    silhouette_avg = silhouette_score(mono_cluster[['PC0_normalized','PC1_normalized','PC2_normalized','PC3_normalized']], cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)
    
    sample_silhouette_values = silhouette_samples(mono_cluster[['PC0_normalized','PC1_normalized','PC2_normalized','PC3_normalized']], cluster_labels)

    y_lower = 10
    
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")


    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(mono_cluster['PC1_normalized'], mono_cluster['PC2_normalized'], 
                marker='.', s=30, lw=0, alpha=0.7,c=colors, edgecolor='k')


    # Labeling the clusters
    centers = kmeans.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
                c="white", alpha=1, s=200, edgecolor='k')

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
                    s=50, edgecolor='k')

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("PC1")
    ax2.set_ylabel("PC2")

    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')

    
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=10) # We define the best number of clusters
clusters = kmeans.fit(mono_cluster[['PC0_normalized','PC1_normalized','PC2_normalized','PC3_normalized']]) #PC1 vs PC2 (normalized values)


In [ ]:
kmeans.cluster_centers_

In [ ]:
mono_cluster['Cluster'] = pd.Series(clusters.labels_, index=df.index)


In [ ]:
mono_cluster

In [ ]:
mono_pca['Cluster'] = pd.Series(clusters.labels_, index=mono_pca.index)
mono_comm = mono_pca[mono_pca['com']==1]
#mono_comm['viz'] = 

In [ ]:
mono_comm.loc[mono_comm['symbol'] == 'Y', 'viz'] = 'ligand'
mono_comm.loc[mono_comm['symbol'] == 'N', 'viz'] = mono_comm['Cluster']


In [ ]:
color_discrete = {"ligand": 'red', 0: 'blue', 1: 'green'}

fig = px.scatter_3d(mono_comm, x='comp_0', y='comp_1', z='comp_2',
                    color = 'viz',
                    #symbol = 'symbol',
                    color_discrete_map=color_discrete,
                    text = 'check',
                    hover_name = 'Kraken ID',
                    size = 'vbur_vbur_min',
                    size_max=30)
#fig.show()
fig.update_layout(margin=dict(l=10, r=10, b=10, t=10))

In [ ]:
!pip install pca

In [ ]:
from pca import pca

In [ ]:
# Initialize model. Alpha is the threshold for the hotellings T2 test to determine outliers in the data.
model = pca(alpha=0.01)

In [ ]:
out = model.fit_transform(df_X)

In [ ]:
ex = out['explained_var']

In [ ]:
exv = out['variance_ratio']

In [ ]:
print(out['outliers'])

In [ ]:
(sns
 .lineplot(x=np.arange(1, 5),
           y=np.cumsum(exv),
           marker="o")
 .set(title="Cumulative PVE vs. Principal Component",
      xlabel="Principal Component",
      ylabel="Cumulative Proportion of Variance Explained"))

In [ ]:
#with plt.style.context('dark_background'):
plt.figure(figsize=(6, 4))

plt.bar(range(4), exv, alpha=0.5, align='center', label='individual explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal components')
plt.legend(loc='best')
plt.tight_layout()

In [ ]:
outlier = pd.DataFrame.from_dict(out['outliers'])
outlier.to_excel('/Users/abhisaurav/Desktop/outliers.xlsx', sheet_name='outliers', index=False)

In [ ]:
#model.biplot(SPE=True,hotellingt2=True,label=None)
model.scatter(SPE=True,hotellingt2=True,label=None)

In [ ]:
# Make biplot
model.biplot(label=None)

In [ ]:
#!pip install rdkit
#!pip install dash

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
from sklearn.cluster import KMeans
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import SimilarityMaps
from rdkit import Chem,Geometry
from rdkit.Chem import rdmolfiles, AllChem, rdMolAlign,rdmolops, Descriptors, Draw, PandasTools
from collections import Counter

In [ ]:
#!pip install jupyter_dash

In [ ]:
# only necessary for the interactive dashboards 

from dash import Dash, dcc, html, Input, Output, State, no_update
import plotly.graph_objects as go
from jupyter_dash import JupyterDash

colors32 = ['#000000', '#222034', '#45283c', '#663931', '#8f563b', '#df7126', 
            '#d9a066', '#eec39a', '#fbf236', '#99e550', '#6abe30', '#37946e', 
            '#4b692f', '#524b24', '#323c39', '#3f3f74', '#306082', '#5b6ee1', 
            '#639bff', '#5fcde4', '#cbdbfc', '#dddddd', '#9badb7', '#847e87', 
            '#696a6a', '#595652', '#76428a', '#ac3232', '#d95763', '#d77bba', 
            '#8f974a', '#8a6f30']

def mol_to_img_df(row):
    mol = Chem.MolFromSmiles(row["smiles"])
    return(Draw.MolToImage(mol))

def dist_to_center(line):
    delta = line[use_cols] - kmeans.cluster_centers_[line["cluster"]]
    dist = np.linalg.norm(delta)
    return(dist)

In [ ]:
features = pd.read_csv('/Users/abhisaurav/Desktop/comm_monophosphine_dash.csv')

print("Features")
print(features.shape)
display(features.head())

In [ ]:
filters = pd.read_csv('/Users/abhisaurav/Desktop/comm_monophosphine_dash.csv')

print("Filters")
print(filters.shape)
display(filters.head())

In [ ]:
use_rows = features.index 
criterion1 = set(filters.loc[filters["com"] == 1].index)
criterion2 = set(filters.loc[filters["com"] == 1].index)
use_rows = sorted(criterion1 & criterion2)                
print(f"Keeping {len(use_rows)} compounds.")

Clustering

In [ ]:
nclusters = 32
use_space = "PCA"

find_cols = {"PCA": ["PC0_normalized","PC1_normalized","PC2_normalized","PC3_normalized"]}

use_cols = find_cols[use_space]
X_kmeans = features.loc[use_rows,use_cols]

kmeans = KMeans(n_clusters=nclusters, random_state=1).fit(X_kmeans)
data_df = features.loc[use_rows].copy()

data_df["closest_to_center"] = 0
data_df["cluster"] = kmeans.labels_
data_df["distance_to_center"] = data_df.apply(dist_to_center, axis = 1)
clustermins = []
for clusternumber in range(nclusters):
    min_to_center_idx = data_df[data_df["cluster"] == clusternumber]["distance_to_center"].idxmin()
    clustermins.append(min_to_center_idx)
    data_df.loc[min_to_center_idx,"closest_to_center"] = 1

clustercounts = Counter(data_df["cluster"])
print("cluster: N(samples)")
for i in sorted(clustercounts.keys()):
    print(f"{i}: {clustercounts[i]}")


In [ ]:
data_df.cluster = data_df.cluster.astype(str)
data_df

Visualisation and Screening

In [ ]:
molobs = [Chem.MolFromSmiles(i) for i in data_df.loc[clustermins]["smiles"]]
legends = [f"Cluster: {k}, ID: {v}" for k,v in enumerate(clustermins)]
img_src = Draw.MolsToGridImage(molobs, subImgSize=(225,250), molsPerRow=4,
                                  legends = legends)
Draw.MolsToGridImage(molobs, subImgSize=(225,250), molsPerRow=5,
                                  legends = legends)

In [ ]:
import numpy as np 
import pandas as pd
import plotly.express as px
# For plotting with matplotlib
import matplotlib.pyplot as plt
# For plotting with seaborn
import seaborn as sns  

color_discrete = {"N": 'green', "Y": 'red'}

fig = px.scatter_3d(data_df, x='comp_0', y='comp_1', z='comp_2',
                    color = 'cluster',
                    #symbol = 'symbol',
                    color_discrete_map=color_discrete,
                    text = 'check',
                    size_max=30,
                   opacity=0.8)
#fig.show()
fig.update_layout(margin=dict(l=10, r=10, b=10, t=10))

In [ ]:
#Reference: Dr. Tobias Gensch, 2022
#Gensch, T.; Smith, S.; Colacot, T.; Timsina, Y.; Xu, G.; Glasspoole, B.; Sigman, 
#M. Design and Application of a Screening Set for Monophosphine Ligands in Metal Catalysis

app = JupyterDash(__name__)

app.layout = html.Div([  
    html.Div("Select cluster:"),
    html.Div(
        dcc.RadioItems(
            id = 'clusters_select',
            options = [{'value': x, 'label': x}
            for x in range(nclusters)],
            value = 0,
            labelStyle={"display": "inline-block"},
    ),
        style = {"width": "60%", "display":"inline-block"}
    ),
    
    html.P(),
    
    html.Div(["Show up to N molecules nearest to the cluster center: ",
        dcc.Input(
            id = "show_n_mols", type = "number", value = 4, min = 1, step = 1,
            style={"width":"5%"}
        )
    ]),
    
    html.Img(id="molecules_img"),
])

@app.callback(Output("molecules_img", "src"),[Input("show_n_mols","value"),Input("clusters_select","value")])

def generate_molimg(show_n_mols,clusters_select):
    show_n_mols = int(show_n_mols)
    clustermin_idx = clustermins[clusters_select]
    
    sorted_mols = data_df.loc[data_df["cluster"] == clusters_select].sort_values(by="distance_to_center")
    showno = min(show_n_mols,len(sorted_mols))
    show_idxs = sorted_mols.iloc[:showno].index
    molobs = [Chem.MolFromSmiles(i) for i in sorted_mols.loc[show_idxs,"smiles"]]
    legends = [f"ID: {i}, Dist: {data_df.loc[i,'distance_to_center']:.2f}" for i in show_idxs]
    img_src = Draw.MolsToGridImage(molobs, subImgSize=(125,150), molsPerRow=3,legends = legends)
    
    return(img_src)

#app.run_server(mode='inline', port = 3689)